# Code to produce the cutouts for the MIRI data

Let's start with the imports

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS, FITSFixedWarning
from astropy.nddata import Cutout2D
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import warnings
from scipy.ndimage import zoom

warnings.simplefilter("ignore", category=FITSFixedWarning)



Let's look at the NIRCam documentation:

So we find that in the short filters we have 0.031"/pix and Amir used 167 x 167 pixels, which means he is using an image size of 5.177" for each cutout. We also know that the total FOV is around 63.74" x 63.74". Now we need to translate this to MIRI:

So taking this conversion of pixels to angular resolution we find that MIRI pixels are on average about 3.5 times larger than NIRCam pixels, so we just need to divide 167 by 3.55, which leaves us with ~48 pixels in MIRI.



# Define function to produce cutouts automatically

# Produce cutouts while preserving all other layers (in the same patch of the sky)

In [ ]:
def produce_cutouts_for_photometry(cat, indir, output_dir, survey, x_pixels, filter, obs="", nan_thresh=0.4, suffix=''):
    """Produces cutouts for all image extensions in the FITS file, preserving each layer."""

    # Load target catalogue
    with fits.open(cat) as catalog_hdul:
        cat_data = catalog_hdul[1].data
        ids = cat_data['id']
        ra = cat_data['ra']
        dec = cat_data['dec']  

    filter_l = filter.lower()
    fits_files = glob.glob(os.path.join(indir, f"*{filter_l}*.fits"))
    print(f"Found {len(fits_files)} FITS files from the {survey} survey with filter {filter}.")
    print("Processing:")
    for f in fits_files:
        print(f"{f}")

    os.makedirs(output_dir, exist_ok=True)
    print(f"Files will be saved to {output_dir}.")
    
    counts = 0
    total = len(ra)
    cutout_size = (x_pixels, x_pixels)

    for fits_file in fits_files:
        with fits.open(fits_file) as hdul:
            # Assume extension 1 contains the WCS for the cutout reference
            ref_data = hdul[1].data
            ref_header = hdul[1].header
            ref_wcs = WCS(ref_header)

            # Loop over galaxies
            for i in range(total):
                target_coord = SkyCoord(ra[i], dec[i], unit=(u.deg, u.deg))

                # Check if inside the field of view
                try:
                    x, y = ref_wcs.world_to_pixel(target_coord)
                except Exception:
                    continue

                if not (0 <= x < ref_data.shape[1] and 0 <= y < ref_data.shape[0]):
                    continue

                cutout_hdul = fits.HDUList()
                cutout_hdul.append(fits.PrimaryHDU(header=hdul[0].header))

                valid_cutout = True
                max_nan_ratio = 0.0

                for ext in range(1, len(hdul)):
                    hdu = hdul[ext]
                    if hdu.data is None or hdu.data.ndim != 2:
                        continue  # Skip non-image or 1D extensions

                    # WCS can vary by extension
                    try:
                        wcs = WCS(hdu.header)
                        cutout = Cutout2D(hdu.data, target_coord, cutout_size, wcs=wcs, mode="partial")
                        cutout_header = cutout.wcs.to_header()
                    except Exception:
                        # If WCS fails, just use the same pixel cutout
                        cutout = Cutout2D(hdu.data, (x, y), cutout_size, mode="partial")
                        cutout_header = hdu.header.copy()

                    nan_ratio = np.isnan(cutout.data).sum() / cutout.data.size
                    max_nan_ratio = max(max_nan_ratio, nan_ratio)

                    cutout_hdu = fits.ImageHDU(data=cutout.data, header=cutout_header)
                    cutout_hdul.append(cutout_hdu)

                if max_nan_ratio < nan_thresh and len(cutout_hdul) > 1:
                    # Save PNG preview from extension 1
                    preview_data = cutout_hdul[1].data
                    plt.figure(figsize=(6, 6))
                    plt.imshow(preview_data, origin="lower", cmap="gray")
                    plt.colorbar()
                    plt.title(f'{survey}: {ids[i]} at {filter}')
                    png_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}{obs}{suffix}.png")
                    plt.savefig(png_filename)
                    plt.close()

                    # Save multi-extension FITS cutout
                    fits_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_{survey}{obs}{suffix}.fits")
                    cutout_hdul.writeto(fits_filename, overwrite=True)
                    counts += 1

    print(f"Produced cutouts for {counts} of {total} galaxies in the catalogue.")


# Create cutouts of the unmodified FITS files

Shifted data:

For the photometry we actually wanna use 8x8 arcsec cutouts for better noise statistics so that we increase the number of pixels from 48x48 to 74x74

In [ ]:
# Define directories
primer003 =  '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER_003/'
primer004 =  '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER_004/'
cweb1 =   '/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web_1/'
cweb2 =   '/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web_2/'

catalogue =  '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts/'

# Produce cutouts for all surveys8700
produce_cutouts_for_photometry(catalogue, primer003, output_dir, 'primer', 74, 'F770W', '003', nan_thresh=0.5)
produce_cutouts_for_photometry(catalogue, primer003, output_dir, 'primer', 74, 'F1800W', '003', nan_thresh=0.5)

produce_cutouts_for_photometry(catalogue, primer004, output_dir, 'primer', 74, 'F770W', '004', nan_thresh=0.5)
produce_cutouts_for_photometry(catalogue, primer004, output_dir, 'primer', 74, 'F1800W', '004', nan_thresh=0.5)

produce_cutouts_for_photometry(catalogue, cweb1, output_dir, 'cweb', 74, 'F770W', '1', nan_thresh=0.5)
produce_cutouts_for_photometry(catalogue, cweb2, output_dir, 'cweb', 74, 'F770W', '2', nan_thresh=0.5)
